In [1]:
from sim import *

import numpy as np

In [37]:
results = simulate(numColleges=2, 
    numStudents=10, 
    spots=3, is_modified_match=True)
print(results)

(defaultdict(<function deferred_acceptance.<locals>.<lambda> at 0x000001785E680860>, {3: 0, 6: 0, 0: 0, 8: 1, 4: 1, 1: 1, 7: 1}), defaultdict(<function deferred_acceptance.<locals>.<lambda> at 0x000001785E6834C0>, {0: [3, 6, 0], 1: [4, 1, 7]}))
